# Seeing the trees for the forest

- https://adventofcode.com/2022/day/8

As often the case, it's best to look for the _opposite_ condition, so what trees are _not_ visible?

This is a typical job for numpy arrays; sweep a line in each of the 4 directions to track the max height seen up to that point, and the trees on the next line are visible if they are taller than the max height seen. Because we need to check for visibility from 4 directions, we start by giving all the trees a score of 4, subtract 1 for trees obscured in the current direction, and after the 4 scans count how many non-zero values are left.

For an forest of size $n$ trees, this takes $O(n)$ time (so, linear).


In [1]:
from dataclasses import dataclass
from typing import Self

import numpy as np


@dataclass
class Forest:
    trees: np.ndarray[np.uint8]

    @classmethod
    def from_text(cls, text: str) -> Self:
        return cls(np.genfromtxt(text.splitlines(), delimiter=1, dtype=np.uint8))

    @property
    def visible(self) -> int:
        """Count the number of trees visible from direct lines of sight."""

        visible = np.full_like(self.trees, 4)
        trees = self.trees
        for _ in range(4):
            trees, visible = np.rot90(trees), np.rot90(visible)
            treeheight = trees[0]
            for tree_row, vis_row in zip(trees[1:], visible[1:]):
                vis_row[tree_row <= treeheight] -= 1
                treeheight = np.maximum(treeheight, tree_row)
        return (visible > 0).sum()


example = """\
30373
25512
65332
33549
35390
"""
assert Forest.from_text(example).visible == 21

In [2]:
import aocd

mapdata = aocd.get_data(day=8, year=2022)

print("Part 1:", Forest.from_text(mapdata).visible)

Part 1: 1672


## Part 2, counting adjacent lower trees

For part 2, you essentially have to count the number of trees in any given direction are _lower_ than that tree, plus 1 if not reaching the edge.

A few things to note:

- The trees at the edges have a score of 0, because they have at least one direction that scores 0. The distances are multiplied, and multiplying by 0 always gives 0.
- Any tree not at the edge will score _at least_ 1: if the trees in all directions are of the same height or higher, it scores $1 \times 1 \times 1 \times 1 = 1$.
- When looking at a row of length $n$, we can represent the distance to the nearest tree of height $h$ in an array of size $10 \times n$, which are incremented by 1 each time you move to the next row to process. You can then set the distances of trees with height $h$ _and lower_ to zero (those are obscured by the taller tree). This way we can then update the distances for the next row without having to scan back to already-processed rows.

This can perhaps best be explained with an example, tracking just the trees in one column; from the example the right-most column has the tree heights 3, 2, 2, 9 and 0. Starting at the edge, the distance array for all heights is set to `(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)` (10 zeros), and after the first step, those are _all_ incremented to 1.

- The second tree has height 2, and the `distance[2]` value is 1, so that's the number of trees it can see looking up. We then set the distance for trees height 0 through to 2 to zero, and then increment all values in the array by one. Now we have `(1, 1, 1, 2, 2, 2, 2, 2, 2, 2)`, meaning, that if the next tree we look at has height 3 through to 9, it could see 2 trees, otherwise, you can see 1 tree.
- The next tree height is 2, so it can see 1 tree looking north. Like before, we set height indices 0 through to 2 to zero, and increment all distances again, giving us `(1, 1, 1, 3, 3, 3, 3, 3, 3, 3)`. We now have a tree of height 9, so it can see 3 trees looking up. Setting the distances for heights 0 through 9 to zero and then incrementing all distances brings us back to all 1s.
- The last tree has height 0, and so can see 1 tree looking up.

This too lets us find the heighest scenic value in $O(n)$ (linear) time.


In [3]:
class ScenicForest(Forest):
    @property
    def max_scenic_score(self) -> int:
        trees = self.trees
        scenic_score = np.ones_like(trees, dtype=np.uint)

        for _ in range(4):
            trees, scenic_score = np.rot90(trees), np.rot90(scenic_score)
            distances = np.zeros((10, trees.shape[1]), np.uint)
            heights = np.repeat(
                np.arange(10, dtype=np.uint8)[:, None], distances.shape[1], axis=1
            )
            for tree_row, score_row in zip(trees, scenic_score):
                score_row *= tree_row.choose(distances)
                distances[heights <= tree_row] = 0
                distances += 1

        return np.max(scenic_score)


assert ScenicForest.from_text(example).max_scenic_score == 8

In [4]:
print("Part 2:", ScenicForest.from_text(mapdata).max_scenic_score)

Part 2: 327180
